
### Jupyter Notebook in VS Code

From directory with virtual environment in it, use this terminal command to use Jupyter Notebook in VS Code:

jupyter notebook --NotebookApp.allow_origin='*' --NotebookApp.ip='0.0.0.0'

### Select Python Interpreter

Open Command Palette as follows: Ctrl+Shift+P

Then select from VS Code Command Palette: "Python: Select Interpreter" and choose the same Python interpreter that you used to install Jupyter.

If server not shown, enter server manually as follows: 
http://localhost:8888

Reminder:  pw: ""

### Reset and clear all variables if needed - waits for confirmation

In [1]:
# %reset  # clear all variables

### Load Libraries and Define Model Class Identcal to Model Class in Training Notebook 

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

class RecSysModel(nn.Module):
    def __init__(self, n_users, n_movies, n_factors):
        super().__init__()
        self.user_embed = nn.Embedding(n_users, n_factors)
        self.movie_embed = nn.Embedding(n_movies, n_factors)
        self.out = nn.Linear(int(n_factors * 2), 1)

    def forward(self, users, movies):
        user_embeds = self.user_embed(users)
        movie_embeds = self.movie_embed(movies)
        output = torch.cat([user_embeds, movie_embeds], dim=1)
        output = self.out(output)
        return output

In [3]:
# set device to cuda if available, otherwise use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


### Load the MovieLens movies dataset for pairing with the ratings dataset to provide movie titles as recommendations

In [5]:
df_movies = pd.read_csv('../Sandbox/MovieLens/ml-latest-small/movies.csv')

### Load the model and label encoders for users and movies

In [ ]:

# Load the model and label encoders from the trained model .pth file
checkpoint = torch.load('models/movie_rec_sys_r2.pth', weights_only=False)

# Extract the label encoders
lbl_user_loaded = checkpoint['lbl_user']
print(f"lbl_user_loaded: {lbl_user_loaded}")
lbl_movie_loaded = checkpoint['lbl_movie']
print(f"lbl_movie_loaded: {lbl_movie_loaded}")
user_embed = checkpoint['user_embed']
print(f"user_embed: {user_embed}")
movie_embed = checkpoint['movie_embed']
print(f"movie_embed: {movie_embed}")
n_factors = checkpoint['n_factors']
print(f"n_factors: {n_factors}")

# Define the model architecture
n_users = len(lbl_user_loaded.classes_)
n_movies = len(lbl_movie_loaded.classes_)

model_loaded = RecSysModel(n_users, n_movies, n_factors)

# Load the model's state dictionary
model_loaded.load_state_dict(checkpoint['model_state_dict'])

# Set the device to use for inference
# model_loaded.to(device)

# Set the model to evaluation mode
model_loaded.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'models/movie_rec_sys_r1.pth'

### Recommend the top K movies for a given user

In [21]:
# Assuming the model is already defined and trained
# model = RecSysModel(n_users, n_movies, n_factors)


def recommend_movies_for_new_user(model, user_ids, movie_ids, top_k=10):
    # Get all movie ids
    movie_ids = torch.LongTensor(range(len(lbl_movie_loaded.classes_)))
    print(f'movie_ids device: {movie_ids.device}')
    print(f'movie_ids: {movie_ids}')
    # Use a placeholder user id (e.g., 0)
    user_id = 0
    user_ids = torch.LongTensor([user_id] * len(lbl_movie_loaded.classes_))
    print(f'user_ids device: {user_ids.device}')
    print(f'user_ids: {user_ids}')
    # Get predictions for all movies
    with torch.no_grad():
        all_predictions = model(user_ids, movie_ids)
        print()
        print(f'all_predictions device: {all_predictions.device}')
        print(f"all_predictions shape: {all_predictions.shape}")
        print(f'all_predictions: {all_predictions}')
    # Ensure top_k does not exceed the number of available movies
    top_k = min(top_k, len(all_predictions))
    print()
    print(f'top_k: {top_k}')
    print(f'len(all_predictions): {len(all_predictions)}')
    # Get top k movie predictions
    # top_k_predictions, top_k_indices = all_predictions.topk(top_k)
    top_k_predictions, top_k_indices = torch.topk(all_predictions, top_k, dim=0, largest=True, sorted=True)
    print()
    print(f'top_k_predictions device: {top_k_predictions.device}')
    print(f'top_k_predictions: {top_k_predictions}')
    print(f'top_k_indices device: {top_k_indices.device}')
    print(f'top_k_indices: {top_k_indices}')
    return top_k_predictions, top_k_indices


### Define ratings for a new user and generate recommendations

In [22]:
# Define ratings for a new user to be used for recommendations

new_user_ratings = {
    'movieId': [2959, 318, 1213, 1221, 858, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],  # Example movie IDs
    'rating': [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   # Example ratings
}

# new_user_ratings = {
#     'movieId': [4896, 5816, 8368, 40815, 54001, 69844, 318], # Harry Potter Movies
#     'rating': [5, 5, 5, 5, 5, 5, 0]
# }

# Convert the new user ratings to a DataFrame
df_new_user_ratings = pd.DataFrame(new_user_ratings)
print(f'MovieId df_new_user_ratings.head(): {df_new_user_ratings.head()}')
# Encode the movie IDs using the loaded label encoder
df_new_user_ratings['movieId'] = lbl_movie_loaded.transform(df_new_user_ratings['movieId'])
print(f'Movie_label df_new_user_ratings.head(): {df_new_user_ratings.head()}')

# Encode the user ID using the loaded label encoder
user_id = 0  # Placeholder user ID

# Add unseen user_id to lbl_user_loaded
if user_id not in lbl_user_loaded.classes_:
    lbl_user_loaded.classes_ = np.append(lbl_user_loaded.classes_, user_id)

df_new_user_ratings['userId'] = lbl_user_loaded.transform([user_id] * len(df_new_user_ratings))
# print(f'df_new_user_ratings: {df_new_user_ratings}')

# Convert the DataFrame to tensors
user_tensor = torch.LongTensor(df_new_user_ratings['userId'].values)
print(f'user_tensor: {user_tensor}')
movie_tensor = torch.LongTensor(df_new_user_ratings['movieId'].values)
print(f'movie_tensor: {movie_tensor}')
# Convert ratings to tensor
ratings_tensor = torch.FloatTensor(df_new_user_ratings['rating'].values)
print(f'ratings_tensor: {ratings_tensor}')

# Get recommendations for the new user by passing the user and movie tensors to the model
top_k_predictions, top_k_indices = recommend_movies_for_new_user(model_loaded, user_tensor, movie_tensor, top_k=100)
print(f'top_k_predictions: {top_k_predictions}')
print(f'top_k_indices: {top_k_indices}')

# Convert top_k_indices to movie IDs
top_k_movie_ids = lbl_movie_loaded.inverse_transform(top_k_indices.numpy())
print(f'top_k_movie_ids: {top_k_movie_ids}')

# Ensure the top_k_movie_ids and top_k_predictions are 1D tensors
print(f"top_k_movie_ids shape: {top_k_movie_ids.shape}")
print(f"top_k_predictions shape: {top_k_predictions.shape}")
top_k_movie_ids = top_k_movie_ids.flatten() if len(top_k_movie_ids.shape) > 1 else top_k_movie_ids
top_k_predictions = top_k_predictions.flatten() if len(top_k_predictions.shape) > 1 else top_k_predictions
print(f"top_k_movie_ids shape: {top_k_movie_ids.shape}")
print(f"top_k_predictions shape: {top_k_predictions.shape}")

# Convert top_k_movie_ids to DataFrame
df_recommendations = pd.DataFrame({
    'movieId': top_k_movie_ids,
    'predicted_rating': top_k_predictions.numpy()
})
print(f'df_recommendations: {df_recommendations}')
# Merge with movie titles
df_recommendations = df_recommendations.merge(df_movies, on='movieId', how='left')
print(f'df_recommendations: {df_recommendations}')

MovieId df_new_user_ratings.head():    movieId  rating
0     2959       0
1      318       0
2     1213       0
3     1221       1
4      858       1
Movie_label df_new_user_ratings.head():    movieId  rating
0     2224       0
1      277       0
2      913       0
3      921       1
4      659       1
user_tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
movie_tensor: tensor([2224,  277,  913,  921,  659,    5,    6,    7,    8,    9,   10,   11,
          12,   13,   14])
ratings_tensor: tensor([0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
movie_ids device: cpu
movie_ids: tensor([   0,    1,    2,  ..., 9721, 9722, 9723])
user_ids device: cpu
user_ids: tensor([0, 0, 0,  ..., 0, 0, 0])

all_predictions device: cpu
all_predictions shape: torch.Size([9724, 1])
all_predictions: tensor([[4.6344],
        [4.1592],
        [3.7284],
        ...,
        [4.6350],
        [3.8800],
        [3.9286]])

top_k: 100
len(all_predictions): 9724

top_k_predictions devi

/home/mbuhidar/Code/mbuhidar/movie_recommender_system/venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Display the top-k recommended movies

In [23]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the top-k recommended movies
print("Top-k recommended movies:")
print(df_recommendations[['movieId', 'title', 'predicted_rating']].head(20))
# df_recommendations

Top-k recommended movies:
    movieId                                              title  \
0       318                   Shawshank Redemption, The (1994)   
1      2019        Seven Samurai (Shichinin no samurai) (1954)   
2       296                                Pulp Fiction (1994)   
3       608                                       Fargo (1996)   
4      4973  Amelie (Fabuleux destin d'Amélie Poulain, Le) ...   
5        50                         Usual Suspects, The (1995)   
6      2959                                  Fight Club (1999)   
7      6869                        Bus 174 (Ônibus 174) (2002)   
8      2329                          American History X (1998)   
9      5307                     Taking Care of Business (1990)   
10    55278                                      Sleuth (2007)   
11       47                        Seven (a.k.a. Se7en) (1995)   
12     3240                              Big Tease, The (1999)   
13      858                              Godfather